In [ ]:
import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import numpy as np
import tensorflow as tf

physical_devices = tf.config.list_physical_devices("GPU")
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, enable=True)
from tensorflow.keras.layers import BatchNormalization

# from keras.layers.normalization import layer_normalization
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))

In [ ]:
N_train = 1000
X = np.expand_dims(np.zeros((1200, 20), dtype=float), axis=2)
Y = np.expand_dims(np.zeros((1200), dtype=float), axis=1)
# Y = np.zeros(1200,dtype=float)
X_train = X[:1000, :]
Y_train = Y[:1000, :]
X_test = X[1000:, :]
Y_test = Y[1000:, :]

In [ ]:
from numpy.random import choice

one_indexes = choice(a=N_train, size=int(N_train / 2), replace=False)
X_train[one_indexes, 0, :] = 1  # very long term memory.
Y_train[one_indexes, :] = 1  # very long term memory.

one_indexes = choice(a=200, size=int(200 / 2), replace=False)
X_test[one_indexes, 0, :] = 1  # very long term memory.
Y_test[one_indexes] = 1  # very long term memory.

In [ ]:
def prepare_sequences(x_train, y_train, window_length):
    windows = []
    windows_y = []
    for i, sequence in enumerate(x_train):
        len_seq = len(sequence)
        for window_start in range(0, len_seq - window_length + 1):
            window_end = window_start + window_length
            window = sequence[window_start:window_end]
            windows.append(window)
            windows_y.append(y_train[i])
    return np.array(windows), np.array(windows_y)

In [ ]:
x_train, y_train = prepare_sequences(X_train, Y_train, 10)
x_test, y_test = prepare_sequences(X_test, Y_test, 10)

In [ ]:
max_len = 10
batch_size = 64
print("Building STATELESS model...")
model = Sequential()
model.add(LSTM(10, input_shape=(max_len, 1), return_sequences=False, stateful=False))
# model.add(LSTM(10, batch_input_shape=(batch_size, max_len, 1), return_sequences=False, stateful=False))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# train_size = batch_size * x_train.shape

In [ ]:
# x_train1 = x_train[:170*batch_size,:,:]
# y_train1 = y_train[:170*batch_size,:]
# x_test1 = x_test[:33*batch_size,:,:]
# y_test1 = y_test[:33*batch_size,:]
#
# model.fit(x_train1, y_train1, batch_size=batch_size, epochs=15,
#           validation_data=(x_test1, y_test1), shuffle=False)
model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=15,
    validation_data=(x_test, y_test),
    shuffle=False,
)

In [ ]:
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=0)